# Computing synthetic seismograms using GEMINI

## Introductory remarks

If you want to compute waveform synthetics seismograms based on GEMINI solutions of the elastic wave equation, this is the first step you need to take.

GEMINI computes synthetic seismograms in two basic steps: the first one is the computation of Green frequency-wavenumber (FK) spectra using the program <b>computeGreenFKSpectraForSynthetics</b>. Seismograms are calculated in a follow-up step using the program <b>computeSyntheticSeismograms</b>. Here, we deal with the first step.

GEMINI uses externally defined radial nodes where solutions are stored and which at the same time serve as potential source positions. <b>computeGreenFKSpectraForSynthetics</b> provides Green FK spectra at one specific receiver node considering every radial node as a source node. The source type and, in case of a parallel run the processor's rank, are appended to the file's base name.

## Creating a working folder

You should first create a new working folder where you want to run <b>computeGreenFKSpectraForSynthetics</b> from and to which parameter files and output files are written. Doing so helps in organizing things and avoiding inconsistencies between in- and output. If you haven't done so yet, please create that folder and copy this notebook to the new place. Then, close and halt this notebook, switch to the working folder and start the copy from there.

## Creating the parameter file

In the following, it is assumed that you are now in your new working folder. The first thing we do is to create the parameter file for <b>computeGreenFKSpectraForSynthetics</b>. To do this we first append the folder where the Python parameter templates of GEMINI are located to the path variable. Please enter the appropriate folder name.

In [3]:
import sys
sys.path.append('/home/wolle/work_git/geminiUnified/python/')

### Earth model

File name of 1D earth model. This file is in JSON format. It specifies the model properties at radial nodes. Discontinuities are represented by double nodes. Single nodes act as anchor points for a <b>continuous</b> depth variation between two discontinuities. Splines are used for interpolation. Here,an example how it looks.

In [4]:
earth_model = 'ak135q.json'

### External radial nodes

Here you may define the set of radial nodes GEMINI uses as potential source nodes and as places where the Green FK spectra are stored. The length unit is <b>km</b>. Make sure that all radial nodes are above the inner homogeneous sphere of the earth model. Note: Indexing of radial nodes increases with radius not depth. Thus, index 1 corresponds to the bottommost node.

First we define the radius of the earth in km. You may choose a different value, for example when computing wave propagation in a tennis ball.

In [5]:
rearth = 6371.

The nodes are organized into several blocks within which nodes are equidistant.

In [6]:
nblocks = 2

In contrast to the ordering of the nodes, the blocks are ordered from top to bottom. The following integer array specifies the number of nodes within each block:

In [7]:
nnod = '10 10'

The following real array specifies the spacing between the nodes of each block, also ordered from top to bottom. Unit is km.

In [8]:
dr = '3 5'

By default, there is always one node at the surface. However, you may shift this node and with it all other nodes below to greater depths:

In [9]:
shift = 0.0

### Miscellaneous parameters

Maximum length of synthetics to be calculated from GFK-spectra in seconds.
Determines the frequency stepping: df = 1/tlen.

In [10]:
tlen = 512

Distance from which wavenumber stepping is calculated in km
(should be about 10 times the maximum source receiver distance)

In [11]:
xlen = 5000.

Upper frequency limit of Green FK-spectra in Hz 
(should be twice the Nyquist frequency desired for synthetics)

In [12]:
fmax = 0.2

Basename of output files for Green FK-spectra
Program appends <i>'.'+sourcetype+'.'+crank</i>, where sourcetype is either FORCE or MOMENT, and rank is the rank of the processor by which the file was written:

In [13]:
basename = 'gfk'

Source type: set 'FORCE' for single force or 'MOMENT' for moment tensor source

In [14]:
source_type = 'MOMENT'

Mode of attenuation. This replaces the attenuation flag in the earth model file which was used in older versions. Allowed values for this parameter are:<br>
ELASTIC, ATTENUATION_ONLY, DISPERSION_ONLY, ATTENUATION_AND_DISPERSION

In [15]:
attn_mode = 'ATTENUATION_AND_DISPERSION'

Accuracy of computation Reasonable values are $10^{-4}$ to $10^{-6}$. The adaptive step size integration method stops refining step size once the estimated error is below this value.

In [16]:
eps = 1.e-5

We also need an upper limit for the wavenumbers. Since this depends linearly on frequency, we instead give a slowness or inverse wave speed limit $p=k/\omega$ expressed in units of s/km. You may specifiy two different values to save computation time. The first one is for the frequency range $0 < f < f_{max}/2$ and the second one for the range $f_{max}/2 < f < f_{max}$. The 2nd one can be more restrictive because contributions by frequencies beyond $f_{max}/2$ will be diminished by a low-pass filter when tranforming into the time domain. Given a value of $p$, waves travelling with apparent speeds of $1/p$ will miss in the synthetic seismograms. A good first estimate for $p$ is the slowness of the slowest surface wave propagating in the model.

In [17]:
slowness_limit_1 = 0.5
slowness_limit_2 = 0.5

Since a formula $k_{max} = p\omega$ would lead to $k_{max} = 0$ at zero frequency, the wavenumber range is extended by a constant fraction of the one at $f_{max}/2$. Here, you can specify this fraction:

In [18]:
wavenumber_margin_fraction = 0.1

Flag for calculating stresses R,S,T in addition to displacements(true or false)

In [19]:
stressflag = '.false.'

Finally, we need to specify the radial node where the receiver is located (typically at the surface):

In [20]:
rec_node_index = 21

Now we are ready to create the parameter file. First, we generate a Python string containing the entire contents of the file and then we write it to the working folder:

In [21]:
from parfile_gfksyn import create_gfksyn_parfile 

parstring = create_gfksyn_parfile(tlen,xlen,fmax,earth_model,basename,
            source_type,attn_mode,eps,slowness_limit_1,slowness_limit_2,wavenumber_margin_fraction,
            stressflag,rearth,nblocks,nnod,dr,shift,rec_node_index)

Now, we write the new parameter file into the compute folder. First, tell what its name is:

In [22]:
outfile = './'+'parfile_gfk'

In [23]:
with open(outfile,'w') as fd:
    fd.write(parstring)